### All of Statistics - Chapter 8 Exercise 2

Conduct a simulation to compare the various bootstrap confidence interval methods.
Let n = 50 and let T(F) =\int (x − µ)^3dF(x)/σ3 be the skewness.

## Requirements:

- Draw Y1,..., Yn ∼ N(0, 1) and set X_i = e^{Y_i}, i = 1,..., n.
- Construct the three types of bootstrap 95% intervals for T(F) from the data X_1,...,X_n.
- Repeat this whole thing many times and estimate the true coverage of the three intervals.

In [2]:
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

In [10]:
def skewness(sample_):
    n = len(sample_)
    mu = sum(sample_) / n
    var = sum((sample_ - mu)**2) / n
    return sum((sample_ - mu)**3 / (n * var**(3/2)))

### Task: Draw Y1, . . . , Yn ∼ N(0, 1)$$ and set $$X_i = e^{Y_i}, i = 1, . . . , n$$.

In [11]:
n = 50
sample = np.exp(norm.rvs(loc = 0, scale = 1, size = 50))

### Task: Construct the three types of bootstrap 95% intervals for $$T(F)$$ from the data $$X_1, . . . ,X_n$$.

In [13]:
bootstrap_repetitions = 9999
bootstrap_estimations = list()
for i in range(bootstrap_repetitions):
    bootstrap_sample = np.random.choice(sample, size = len(sample), replace = True)
    bootstrap_estimations.append(skewness(bootstrap_sample))
bootstrap_estimations = np.sort(bootstrap_estimations)
skewness_hat = skewness(sample)
se_hat = np.array(bootstrap_estimations).std()
print(f'Plug-in estimate for the skewness: skew_hat = {skewness_hat}')
print(f'Bootstrap std. error estimate = {se_hat}')

alpha = 0.05

# Normal method
z = norm.ppf(1-alpha/2)
normal_upper_bound = skewness_hat + se_hat * z
normal_lower_bound = skewness_hat - se_hat * z
print(f'Normal method CI:({normal_lower_bound}, {normal_upper_bound})')

# Percentile method
percentile_upper_bound = np.quantile(bootstrap_estimations, 1 - alpha/2)
percentile_lower_bound = np.quantile(bootstrap_estimations, alpha/2)
print(f'Percentile method CI: ({percentile_lower_bound}, {percentile_upper_bound})')

# Pivotal method
pivotal_lower_bound = 2*skewness_hat - np.quantile(bootstrap_estimations, 1-alpha/2)
pivotal_upper_bound = 2*skewness_hat - np.quantile(bootstrap_estimations, alpha/2)
print(f'Pivotal method CI: ({pivotal_lower_bound}, {pivotal_upper_bound})')

Plug-in estimate for the skewness: skew_hat = 3.0362632057221073
Bootstrap std. error estimate = 0.7025490656702861
Normal method CI:(1.6592923396360812, 4.413234071808134)
Percentile method CI: (1.7293946139347696, 4.493435131414713)
Pivotal method CI: (1.5790912800295018, 4.343131797509445)


### Task: Repeat this whole thing many times and estimate the true coverage of the three intervals.

In [15]:
num_repeats = 9999
coverage = dict(
    normal = 0,
    percentile = 0,
    pivotal = 0
)

for i in range(num_repeats):
    bootstrap_sample = np.random.choice(sample, size = len(sample), replace = True)
    bootstrap_sample_skewness = skewness(bootstrap_sample)
    
    if normal_lower_bound < bootstrap_sample_skewness < normal_upper_bound:
        coverage['normal'] += 1
    if pivotal_lower_bound < bootstrap_sample_skewness < pivotal_upper_bound:
        coverage['pivotal'] += 1
    if percentile_lower_bound < bootstrap_sample_skewness < percentile_upper_bound:
        coverage['percentile'] += 1
    
coverage['normal'] /= num_repeats
coverage['percentile'] /= num_repeats
coverage['pivotal'] /= num_repeats

for method, coverage_pct in coverage.items():
    print(f'The {method} method has a coverage of {np.round(coverage_pct,4)}%')

The normal method has a coverage of 0.9532%
The percentile method has a coverage of 0.9527%
The pivotal method has a coverage of 0.9542%
